In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [2]:
df = pd.read_csv('attention_shift_events.csv')

# Clean the dataframe of unneeded columns
columns_to_drop = ['Unnamed: 0', 'seconds_after_trigger', 'sub_id']
df.drop(columns_to_drop, axis=1, inplace=True)

# Convert target_finding_results from True/False to 1/0
df['target_finding_result'] = df['target_finding_result'].astype(int)

# Masks
both_mask = df['condition'] == 'Both'
sound_mask = df['condition'] == 'Sound'
visual_mask = df['condition'] == 'Visual'

both_df = df[both_mask]
sound_df = df[sound_mask]
visual_df = df[visual_mask]

In [3]:
df

,target_area,target_sign,target_finding_time,target_finding_result,trial_id,condition
0,0,0,9.207268,1,0,Visual
1,1,1,1.611088,0,0,Visual
2,1,1,2.012726,1,0,Visual
3,1,0,4.534775,1,0,Visual
4,1,1,8.184965,1,0,Both
...,...,...,...,...,...,...
431,1,1,1.646162,1,3,Both
432,1,0,2.048754,1,3,Both
433,0,1,1.213605,0,3,Both
434,1,0,1.690796,1,3,Both


In [11]:
# Two way ANOVA for target finding errors
model = ols('target_finding_result ~ C(target_finding_time) + C(target_area) + C(condition) + C(target_finding_time):C(target_area) + C(condition):C(target_area) + C(target_finding_time):C(condition)', data=df).fit()
table = sm.stats.anova_lm(model, typ=2)

print(table)

/Users/nolanbrady/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 432, but rank is 45
  warnings.warn('covariance of constraints does not have full '
/Users/nolanbrady/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 2, but rank is 0
  warnings.warn('covariance of constraints does not have full '
/Users/nolanbrady/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:1900: RuntimeWarning: invalid value encountered in true_divide
  F /= J
/Users/nolanbrady/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 432, but rank is 221
  warnings.warn('covariance of constraints does not have full '


                                             sum_sq     df             F  \
C(target_finding_time)                 4.792278e+01  432.0  2.091848e+24   
C(target_area)                         8.125074e-12    1.0  1.532143e+14   
C(condition)                                    NaN    2.0           NaN   
C(target_finding_time):C(target_area)  4.721537e+01  432.0  2.060970e+24   
C(condition):C(target_area)            1.223054e+00    2.0  1.153155e+25   
C(target_finding_time):C(condition)    9.707122e+01  864.0  2.118599e+24   
Residual                               5.303078e-26    1.0           NaN   

                                             PR(>F)  
C(target_finding_time)                 5.486083e-13  
C(target_area)                         5.143165e-08  
C(condition)                                    NaN  
C(target_finding_time):C(target_area)  5.551534e-13  
C(condition):C(target_area)            2.082290e-13  
C(target_finding_time):C(condition)    5.476933e-13  
Residual     

/Users/nolanbrady/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 864, but rank is 287
  warnings.warn('covariance of constraints does not have full '


In [28]:
# Two way ANOVA for target finding time
model = ols('target_finding_time ~ C(target_finding_result) + C(target_area) + C(condition) + C(target_finding_result):C(target_area) + C(condition):C(target_area) + C(target_finding_result):C(condition)', data=df).fit()
table = sm.stats.anova_lm(model, typ=2)

print(table)

                                              sum_sq     df          F  \
C(target_finding_result)                    0.115022    1.0   0.007547   
C(target_area)                             15.819523    1.0   1.038017   
C(condition)                               23.829946    2.0   0.781815   
C(target_finding_result):C(target_area)    77.001181    1.0   5.052526   
C(condition):C(target_area)                 9.993912    2.0   0.327881   
C(target_finding_result):C(condition)     319.373766    2.0  10.478049   
Residual                                 6492.297375  426.0        NaN   

                                           PR(>F)  
C(target_finding_result)                 0.930812  
C(target_area)                           0.308861  
C(condition)                             0.458230  
C(target_finding_result):C(target_area)  0.025101  
C(condition):C(target_area)              0.720630  
C(target_finding_result):C(condition)    0.000036  
Residual                                   

In [21]:
# Posthoc
posthoc = pairwise_tukeyhsd(df['target_finding_time'], df['target_area'], alpha=0.05)
print(posthoc)
# Posthoc
print()
posthoc = pairwise_tukeyhsd(df['target_finding_time'], df['condition'], alpha=0.05)
print(posthoc)
# Posthoc
print()
posthoc = pairwise_tukeyhsd(df['target_finding_time'], df['target_finding_result'], alpha=0.05)
print(posthoc)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj  lower  upper  reject
--------------------------------------------------
     0      1   0.6277 0.105 -0.1318 1.3872  False
--------------------------------------------------

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
  Both  Sound   0.7318 0.2787 -0.3943 1.8579  False
  Both Visual   0.3627 0.7186 -0.7381 1.4636  False
 Sound Visual  -0.3691 0.7181 -1.4879 0.7498  False
---------------------------------------------------

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     0      1     0.08 0.8886 -1.0414 1.2014  False
---------------------------------------------------


In [23]:
# Posthoc
posthoc = pairwise_tukeyhsd(df['target_finding_result'], df['target_area'], alpha=0.05)
print(posthoc)
# Posthoc
print()
posthoc = pairwise_tukeyhsd(df['target_finding_result'], df['condition'], alpha=0.05)
print(posthoc)
# Posthoc
print()
posthoc = pairwise_tukeyhsd(df['target_finding_time'], df['target_finding_result'], alpha=0.05)
print(posthoc)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     0      1   0.0324 0.3202 -0.0316 0.0964  False
---------------------------------------------------

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
  Both  Sound   0.0482 0.4562 -0.0466  0.143  False
  Both Visual    0.004 0.9944 -0.0887 0.0966  False
 Sound Visual  -0.0442 0.5117 -0.1384 0.0499  False
---------------------------------------------------

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     0      1     0.08 0.8886 -1.0414 1.2014  False
---------------------------------------------------


In [27]:
# Impacts of Condition
posthoc = pairwise_tukeyhsd(df['target_finding_time'], df['condition'], alpha=0.05)
print(posthoc)
print()
posthoc = pairwise_tukeyhsd(df['target_finding_result'], df['condition'], alpha=0.05)
print(posthoc)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
  Both  Sound   0.7318 0.2787 -0.3943 1.8579  False
  Both Visual   0.3627 0.7186 -0.7381 1.4636  False
 Sound Visual  -0.3691 0.7181 -1.4879 0.7498  False
---------------------------------------------------

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
  Both  Sound   0.0482 0.4562 -0.0466  0.143  False
  Both Visual    0.004 0.9944 -0.0887 0.0966  False
 Sound Visual  -0.0442 0.5117 -0.1384 0.0499  False
---------------------------------------------------


In [43]:
# Test effect between the peripheral and central vision
peripheral_mask = df['target_area'] == 1
central_mask = df['target_area'] == 0

peripheral_df = df[peripheral_mask]
central_df = df[central_mask]

results = sm.stats.anova_oneway([central_df['target_finding_time'], peripheral_df['target_finding_time']], use_var="equal", welch_correction=True)
print(results)
# print()

results = sm.stats.anova_oneway([central_df['target_finding_result'], peripheral_df['target_finding_result']], use_var="equal", welch_correction=True)
# print(results)

t_result = sm.stats.ttest_ind(central_df['target_finding_time'], peripheral_df['target_finding_time'], alternative='two-sided', usevar='pooled', weights=(None, None), value=0)
print(t_result)

statistic = 2.638520035725995
pvalue = 0.1050266612994851
df = (1.0, 434.0)
df_num = 1.0
df_denom = 434.0
nobs_t = 436.0
n_groups = 2
means = [2.90906521 3.53676308]
nobs = [214. 222.]
vars_ = [14.13930093 18.32604365]
use_var = equal
welch_correction = True
tuple = (2.638520035725995, 0.1050266612994851)
(-1.62435218955927, 0.10502666129948073, 434.0)
